In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
client = Client(n_workers=1)
client

In [ ]:
#df = dd.read_parquet('data/train_data_pqt.parquet', columns=['user_id','request_cnt','url_host'])
df = pq.read_table('data/train_data_pqt.parquet',columns=['user_id','request_cnt','url_host'])

In [ ]:
df.compute().info()

In [ ]:
data_target_file = pd.read_parquet('data/public_train.pqt')

data_agg = df.select(['user_id', 'url_host', 'request_cnt'])
data_id_url = data_agg.to_pandas()

data_id_url_sum_male = data_id_url.merge(data_target_file[['is_male','user_id']], on = 'user_id', how = 'inner')
data_id_url_sum_male = data_id_url_sum_male.loc[~(data_id_url_sum_male['is_male'].isna()) & (data_id_url_sum_male['is_male'] != 'NA')]
data_id_url_sum_male['is_male'] = data_id_url_sum_male['is_male'].astype('int8')

data_id_url_sum_male['male_request_cnt'] = data_id_url_sum_male['is_male'] * data_id_url_sum_male['request_cnt']
data_id_url_sum_male = data_id_url_sum_male.groupby('url_host').agg({'request_cnt':'sum','male_request_cnt':'sum'})
data_id_url_sum_male['male_fraction'] = data_id_url_sum_male['male_request_cnt']/data_id_url_sum_male['request_cnt']
display(data_id_url_sum_male.sort_values('request_cnt').tail())
display(data_id_url_sum_male.info())